### 0.准备

In [52]:
import re
import requests
from requests.exceptions import ReadTimeout, ConnectionError, RequestException
import urllib3

In [53]:
# 使用requests访问https时会有SSL验证，需要在get方法时关闭验证，同时会显示警告
urllib3.disable_warnings() # 消除警告

### 1.第一部分，爬取主页面

In [29]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.67 Safari/537.36 Edg/87.0.664.47'}
req = requests.get("http://www.las.cas.cn/", verify=False, timeout=5,headers=headers)

In [32]:
#对结果编码
req.encoding = 'UTF-8'

In [50]:
#提取页面title
title = re.findall(r'title>\n(.*?)\n',req.text,re.S)
print(title[0])

        中国科学院文献情报中心


In [37]:
#提取页面语种
languague = re.findall('lang="(.+)"',req.text)
print(languague)

['zh-CN']


In [51]:
#保存html文件
fileOb = open('las.html','w',encoding='utf-8')     #存入本地
fileOb.write(req.text)
fileOb.close()

### 2.第二部分，深度遍历页面内的子链接并爬虫

In [54]:
visited = [] # 已经爬过的url列表。
unvisited = [] # 已经取出的、但是还没有访问的的url
url_count = 0 # 已经访问过的url数量
END_COUNT = 30 # 总共需要访问的url数量
end_flag = False # url_count >= END_COUNT结束

In [4]:
# 访问页面：提取页面里包含的url
def visit(url, depth):
    visited.append(url) # 当前url加入已爬过列表
    try:
        req = requests.get(url, verify=False, timeout=5) 
        # verify参数：关闭SSL验证
    #处理可能存在的异常情况
    except ReadTimeout: # 超时异常，则把当前的 url 放到队列中，稍微重试
        print('Timeout: ', url)
    except ConnectionError: # 连接异常
        print('Connection error: ', url)
    except RequestException: # 请求异常
        print('Error: ', url)
    else:
        if req.status_code == 404:
            print('404页面不存在: ', url)
            ## 把当前的 url 从爬虫任务中删除掉
        if req.status_code == 403:
            print('403页面禁止访问: ', url)
            ## ... 
        if req.status_code == 200:
            # 如果正确访问，count+1；判断是否要结束
            global url_count
            global end_flag
            url_count += 1
            if url_count >= END_COUNT:
                end_flag = True
            print("\t"*depth, "#%d-%d %s"%(depth, url_count, url))
            PATTERN_URl = "<a.*href=\"(https?://.*?)[\"|\'].*"
            ulist = re.findall(PATTERN_URl, req.text)
            ulist = [url for url in ulist if url.find(".pdf") == -1 ] # 跳过pdf文件
            return ulist
    return None

In [5]:
# 深度遍历
def dfs(url, depth=1):
    ulist = visit(url, depth)
    if ulist:
        ulist = list(set(ulist)-set(visited)) 
        # 局部变量ulist局部变量，指一个节点的所有子节点
        for url in ulist:
            if depth<2 and not end_flag: # 只访问1、2层（根节点是第0层）
                dfs(url, depth+1)


In [55]:
if __name__ == '__main__':
    start_url = "http://www.las.cas.cn/"
    print("\t"*0, "#%d %s"%(0, start_url))
    dfs(start_url)

 #0 http://www.las.cas.cn/
	 #1-1 http://www.las.cas.cn/
Connection error:  https://chaxin.las.ac.cn/
		 #2-2 https://bszs.conac.cn/sitename?method=show&id=08E6211A04C21453E053012819ACFCA4
		 #2-3 http://www.las.cas.cn/jb/
Connection error:  http://stpaper.cn/
		 #2-4 http://www.las.cas.cn/party/
		 #2-5 https://las.arp.cn/
Connection error:  http://whcb.las.ac.cn/
		 #2-6 https://www.cas.cn/zt/hyzt/2020jldh/
		 #2-7 https://www.cas.cn/zt/kjzt/2021nzkjpd/
Connection error:  https://www.las.ac.cn/
		 #2-8 http://sciencechina.cn/
		 #2-9 https://www.cas.cn/zt/kjzt/liangdianshaixuan2021_3/
		 #2-10 http://www.clas.cas.cn/
		 #2-11 http://www.casmooc.cn/
		 #2-12 http://www.whlib.ac.cn/
		 #2-13 https://www.cas.cn/zt/kjzt/cewhcg/
		 #2-14 https://www.cas.cn/zt/hyzt/cas2023gzh/
Connection error:  http://opac.las.ac.cn/
Connection error:  http://workflow.las.ac.cn/
		 #2-15 https://mail.cstnet.cn/
		 #2-16 https://www.cas.cn/zt/kjzt/sswcj/
		 #2-17 http://www.fenqubiao.com/
